In [1]:
import numpy as np
import pandas as pd

In [87]:
drug_map = {'1032': 0, '281': 1, '1021': 2, '186': 3, '150': 4, '190': 5, '1019': 6, '249': 7, '1005': 8, '37': 9, '1006': 10, '1373': 11, '51': 12, '1': 13, '134': 14, '1010': 15, '238': 16, '34': 17, '119': 18, '1020': 19, '1008': 20, '1013': 21, '1017': 22, '1054': 23, '199': 24, '155': 25, '71': 26, '1175': 27, '206': 28, '30': 29, '5': 30, '1259': 31, '1199': 32, '1375': 33, '1372': 34, '1009': 35, '1033': 36, '1012': 37}
drug_severities = pd.read_csv('GDSC_ALL/severity_scores.csv')
drug_severities['drug_id'] = drug_severities['drug_id'].astype(str)
drug_severities['drug_id'] = drug_severities['drug_id'].map(drug_map)
drug_severities = drug_severities[['drug_id', 'normalised']]
drug_severities.set_index('drug_id', inplace=True)
severity_map = drug_severities.to_dict()['normalised']
sorted_severities = sorted(severity_map.items(), key=lambda x: x[1])
sorted_severities

[(32, 2.1707317073170733),
 (23, 2.339468302658487),
 (6, 2.407563025210084),
 (11, 2.4764397905759163),
 (8, 2.489430894308943),
 (12, 2.546875),
 (27, 2.590909090909091),
 (35, 2.595505617977528),
 (20, 2.61036036036036),
 (30, 2.6301369863013697),
 (7, 2.633228840125392),
 (36, 2.68),
 (2, 2.685358255451713),
 (3, 2.7222222222222223),
 (21, 2.7594654788418707),
 (31, 2.7857142857142856),
 (4, 2.814589665653495),
 (29, 2.857549857549857),
 (18, 2.9017857142857144),
 (19, 2.902547065337763),
 (33, 2.9069767441860463),
 (28, 2.9654255319148937),
 (37, 3.0),
 (25, 3.036764705882353),
 (9, 3.0397553516819573),
 (34, 3.042105263157895),
 (10, 3.0583756345177666),
 (24, 3.066666666666667),
 (17, 3.087012987012987),
 (26, 3.2),
 (22, 3.2051282051282053),
 (14, 3.214689265536723),
 (0, 3.2203389830508478),
 (5, 3.243589743589744),
 (1, 3.2454545454545456),
 (16, 3.272251308900524),
 (15, 3.3719512195121952),
 (13, 3.6666666666666665)]

In [104]:
min_sorted_severities = sorted(severity_map.items(), key=lambda x: x[1])
min_sorted_severities = [x[1] for x in min_sorted_severities]
max_sorted_severities = min_sorted_severities[::-1]
indices = np.arange(1,39)

def pen_func(severities):
    log_pens = severities * (-1*np.log(indices)+4)
    total_log_pen = log_pens.sum()
    total_log_pen_norm = total_log_pen / len(severities) # number of drugs isnt always the same
    total_log_pen_norm = total_log_pen_norm * 100
    return total_log_pen_norm

min_pen = pen_func(min_sorted_severities)
max_pen = pen_func(max_sorted_severities)

print('min pen:', min_pen)
print('max pen:', max_pen)

min pen: 346.27648427400703
max pen: 396.88853997040997


### Non constrained files:

In [220]:
# Non constrained files:
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rewardscale100.npz') #.74184 . reached 0.74 NDCG
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100.npz')
#ranking = np.load('rankings/PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100.76029.npz')
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_constrtrue_gradlambdanegative.npz') # this wasnt actually doing any constraints - what I added accidentally acted as a form of regularisation. reached 0.76 NDCG.
ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_0.76ndcg.npz') # from .76029 - reached 0.76 NDCG. atm prefer this one as benchmark for non-constrained.

In [236]:
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_constrainedtrue.npz')
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_constrainedtrue_12hr.npz') # lambda was adjusting properly here. was a small lambda value though, so worked less well than below.
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_12hr_constrtrue.npz') #this has a constantly increasing lambda value. worked well tho
#ranking = np.load('rankings/lambda1Ypred.npz') # top ranked drug was very low, 2nd was actually quite high. 3rd was lower, 4th slightly higher and 5th lower.
#ranking = np.load('rankings/PPORank_FYP_lr2e-4_15xlrsched_epochs700_rwdscl100_12hr_constrtrue_lagrlam0.1.npz')
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_constrtrue_gradlambdaorig.npz') #in the end this had constantly decreasing lambda - only went for 292 epochs though
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_constrtrue_gradlambdanegative_epoch400.npz') #in the end this had constantly increasing lambda. went for 400 epochs.
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_constr_laminupdate_alladds_epoch300.npz')
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_constr_laminupdate_minuspen_epoch300.npz')
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100_constr_laminval_objfuncadd_losspenneg_epoch300.npz')
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.7e-4_30xlrsched_epochs700_constrtrue_lagrlambda20_lambdalr0.2_epoch300.npz')
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.7e-4_30xlrsched_epochs700_constrtrue_lagrlambda40_lambdalr0.2.npz')
#ranking = np.load('rankings/ppo_0_240526_1629_epoch500.npz') #is --lr=1.5e-4 --reward_scale_factor=100 --constrained=True --lagrange_lambda=40 --lambda_lr=0.005
#ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.7e-4_15xlrsched_epochs700_constr_quadraticloss_rho0.5.npz')
#ranking = np.load('rankings/ppo_0_240528_0358.npz')
ranking = np.load('rankings/ppo_0_240528_1517_properlang_macrun.npz')

### Lagrange loss files:

In [277]:
ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_constr_lagrproper_rwdscl1_lagrloss_lam0.1_lamlr1e-4_tgt3.6_CPU_epoch400.npz')

### Quadratic loss files:

In [270]:
ranking = np.load('rankings/ppo_0_PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_constr_rwdscl1_quadloss_rho2.8_tgtpen3.6.npz')

#### things to note

plot_files/PPORank_FYP_lr1.5e-4_15xlrsched_epochs700_rwdscl100.e75847 has the best performance so far.

In [278]:
pred = ranking['Y_pred']
#true = ranking['Y_true']

In [279]:
pred_df = pd.DataFrame(pred)
#true_df = pd.DataFrame(true)

In [280]:
# each row is a cell line, each column is a drug. Drug with highest value is most effective
# get 5 most effective drugs for each cell line

n = 10

def get_top_5_drugs(row):
    return row.nlargest(n).index

top_5_pred = pred_df.apply(get_top_5_drugs, axis=1)
top_5_pred = pd.DataFrame(top_5_pred.tolist(), index=pred_df.index, columns=[i for i in range(1, n+1)])
most_common_per_rank = top_5_pred.mode(axis=0).transpose()
#most_common_per_rank.columns = ['most common drug']

#top_5_true = true_df.apply(get_top_5_drugs, axis=1)
#top_5_true = pd.DataFrame(top_5_true.tolist(), index=true_df.index, columns=[1, 2, 3, 4, 5])

In [281]:
top_5_pred_severities = pd.DataFrame(index=top_5_pred.index, columns=[i for i in range(1, n+1)])
#top_5_true_severities = pd.DataFrame(index=top_5_true.index, columns=[1, 2, 3, 4, 5])
for i in range(1, n+1):
    top_5_pred_severities[i] = top_5_pred[i].map(severity_map)
    #top_5_true_severities[i] = top_5_true[i].map(severity_map)

In [282]:
avg_pred_severities = top_5_pred_severities.mean(axis=0)
#avg_true_severities = top_5_true_severities.mean(axis=0)
avg_pred_severities = pd.DataFrame(avg_pred_severities, columns=['avg pred severity'])
avg_pred_severities=avg_pred_severities.join(most_common_per_rank)
avg_pred_severities.columns = [ 'avg pred severity', 'most common drug']
avg_pred_severities['severity of most common drug'] = avg_pred_severities['most common drug'].map(severity_map)
avg_pred_severities.index.name = 'rank'

In [228]:
avg_pred_severities # leave this cell to be the non-constrained values

,avg pred severity,most common drug,severity of most common drug
rank,,,
1,2.954827,34,3.042105
2,2.769648,12,2.546875
3,2.989299,18,2.901786
4,2.994967,0,3.220339
5,2.953523,28,2.965426
6,2.974240,16,3.272251
7,3.001563,21,2.759465
8,2.919005,21,2.759465
9,2.969353,13,3.666667


In [276]:
avg_pred_severities

,avg pred severity,most common drug,severity of most common drug
rank,,,
1,2.551773,12,2.546875
2,2.898082,34,3.042105
3,2.571645,11,2.476440
4,2.758992,18,2.901786
5,2.661675,32,2.170732
6,2.869786,28,2.965426
7,2.981898,28,2.965426
8,3.008489,16,3.272251
9,2.740006,30,2.630137


In [283]:
avg_pred_severities

,avg pred severity,most common drug,severity of most common drug
rank,,,
1,2.546875,12,2.546875
2,3.042105,34,3.042105
3,2.196166,32,2.170732
4,2.801986,4,2.814590
5,2.485953,11,2.476440
6,2.662814,7,2.633229
7,2.808489,21,2.759465
8,3.115955,0,3.220339
9,2.868612,18,2.901786
